In [ ]:
!pip install keras-swa==0.1.5

In [ ]:
from swa.keras import SWA
import tensorflow.keras.backend as K

In [ ]:
from keras.datasets import cifar10

In [ ]:
(trainX, trainy), (testX, testy) = cifar10.load_data()
trainX = trainX/255
testX = testX/255

In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
from keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.fit_transform(testy)

In [ ]:
def label_dict(img, labels):
  lab_lis = {}
  for i in range(10):
    lab_lis[i] = []
  data = list(zip(img, labels))
  for im, lab in data:
    idx = np.argmax(lab)
    lab_lis[idx].append((im, lab))
  return lab_lis

In [ ]:
t = label_dict(trainX, trainy)

In [ ]:
unlab_data = []
unlab_img_data = []
for i in range(len(t)):
  unlab_data.extend(t[i][4000:])
for x,y in unlab_data:
  unlab_img_data.append(x)

In [ ]:
unlab_img_data[0]

array([[[0.99607843, 0.99607843, 0.99607843],
        [0.98823529, 0.98823529, 0.98823529],
        [0.98823529, 0.98823529, 0.98431373],
        ...,
        [0.98823529, 0.98823529, 0.99215686],
        [0.98823529, 0.98823529, 0.98039216],
        [1.        , 1.        , 0.99607843]],

       [[1.        , 1.        , 1.        ],
        [0.99607843, 0.99607843, 0.99607843],
        [0.99607843, 0.99607843, 0.98823529],
        ...,
        [0.99607843, 0.99607843, 1.        ],
        [0.99607843, 0.99607843, 0.98823529],
        [1.        , 1.        , 1.        ]],

       [[1.        , 1.        , 1.        ],
        [0.99607843, 0.99607843, 0.99607843],
        [1.        , 0.99607843, 0.99215686],
        ...,
        [0.99607843, 0.99607843, 1.        ],
        [0.99607843, 0.99607843, 0.98823529],
        [1.        , 1.        , 1.        ]],

       ...,

       [[0.96862745, 0.94509804, 0.95294118],
        [0.96078431, 0.93333333, 0.94117647],
        [0.96470588, 0

In [ ]:
client_names = ["client_"+str(i) for i in range(10)]
client_data = {}
for client in client_names:
  client_data[client] = []
i = 0
for client in client_names:
  client_data[client].extend(t[i%10][:1960])
  client_data[client].extend(t[(i+1)%10][1960:3920])
  i += 1
for i in range(10):
  k = 0
  for j in range(10):
    if j != (i)%10 and j != (i+1)%10:
      client_data["client_"+str(j)].extend(t[i][3920 + 10*k: 3920 + 10*(k+1)])
      k += 1

In [ ]:
def batch(zip_data, batch_size = 40):
  data, label = zip(*zip_data)
  dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
  return dataset.shuffle(len(label)).batch(batch_size)

In [ ]:
client_batched = {}
for (name, data) in client_data.items():
  client_batched[name] = batch(data)

test_batched = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(len(testy))
unlab_dataset = tf.data.Dataset.from_tensor_slices((unlab_img_data)).batch(len(unlab_img_data))

In [ ]:
class MLP:
    @staticmethod
    def build():
        tf_input = Input(shape=(32, 32,  3))
        base_model = ResNet50(input_tensor=tf_input, include_top=False)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        predictions = Dense(10, activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=predictions)
        return model

In [ ]:
model = MLP()
model = model.build()

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
t = model.get_weights()
for v in t:
  print(np.shape(v))

(7, 7, 3, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 256)
(256,)
(1, 1, 64, 256)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(1, 1, 256, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 256)
(256,)
(256,)
(256,)
(256,)
(256,)
(1, 1, 256, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(64,)
(64,)
(1, 1, 64, 256)
(256,)
(256,)
(256,)
(256,)
(256,)
(1, 1, 256, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(1, 1, 256, 512)
(512,)
(1, 1, 128, 512)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(1, 1, 512, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(1, 1, 128, 512)
(512,)
(512,)
(512,)
(512,)
(512,)
(1, 1, 512, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(1, 1, 128, 

In [ ]:
comms_round = 50
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=0.01, 
                momentum=0.9
               )   
optimizer2 = SGD(lr=0.001,
                 momentum=0.9
               )   

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun May 16 16:13:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    33W /  70W |    478MiB / 15109MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# model = MLP()
# model = model.build()
# model.compile(loss=loss, 
#                       optimizer=optimizer2, 
#                       metrics=metrics)
# print((model.optimizer.lr))
# model2 = MLP()
# model2 = model2.build()
# model2.compile(loss=loss, 
#                       optimizer=optimizer, 
#                       metrics=metrics)
# print((model2.optimizer.lr))

In [ ]:
weight_scale = []
for (name, data) in client_data.items():
  weight_scale.append(len(data)/40000)
weight_scale = np.array(weight_scale).reshape((len(weight_scale), 1))

In [ ]:
# def scale_model_weights(weight, scalar):
#     weight_final = []
#     steps = len(weight)
#     for i in range(steps):
#         weight_final.append(scalar * weight[i])
#     return weight_final

def sum_scaled_weights(scaled_weight_list, M, unlab_data):
    #print(type(scaled_weight_list[0][0]))
    temp = MLP()
    model = temp.build()
    p = np.zeros((10000,10))
    for i in range(len(weight_scale)):
      model.set_weights(scaled_weight_list[i])
      p += model.predict(unlab_data)
    avg_grad_mean = list()
    avg_grad_var = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        #print(np.shape(grad_list_tuple))
        if len(np.shape(grad_list_tuple)) == 5:
          w = weight_scale.reshape((len(weight_scale), 1, 1, 1, 1))
        elif len(np.shape(grad_list_tuple)) == 4:
          w = weight_scale.reshape((len(weight_scale), 1, 1, 1))
        elif len(np.shape(grad_list_tuple)) == 3:
          w = weight_scale.reshape((len(weight_scale), 1, 1))
        else:
          w = weight_scale
        layer_mean = np.sum(grad_list_tuple*w, axis=0)
        #print(np.shape(layer_mean))
        layer_variance = np.sum(np.power(grad_list_tuple - layer_mean, 2)*w, axis = 0) 
        avg_grad_mean.append(layer_mean)
        avg_grad_var.append(layer_variance)
    model.set_weights(avg_grad_mean)
    p += model.predict(unlab_data)
    for i in range(M):
      grad_list = list()
      for j in range(len(avg_grad_var)):
        grad_list.append(np.random.normal(avg_grad_mean[j], avg_grad_var[j]))
      model.set_weights(grad_list)
      p += model.predict(unlab_data)
    return p/(M + len(weight_scale) + 1), avg_grad_mean


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
u_lab = np.stack(unlab_img_data)

In [ ]:
mlp_global = MLP()
global_model = mlp_global.build()

for comm_round in range(comms_round):
            
    
    global_weights = global_model.get_weights()
    
    
    scaled_local_weight_list = list()
    client_names = list(client_batched.keys())
    random.shuffle(client_names)
    
    for client in client_names:
        def scheduler(epoch, lr):
            if epoch  == 8:
              lr = lr*0.1
            if epoch == 15:
              lr = lr*0.1  
            return lr
        mlp_local = MLP()
        local_model = mlp_local.build()
        local_model.compile(loss=loss, 
                      optimizer=SGD(lr=0.01,momentum=0.9), 
                      metrics=metrics)
        local_model.set_weights(global_weights)
        
        callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        local_model.fit(client_batched[client], epochs=5, verbose=0, callbacks=[callback])
        scaled_local_weight_list.append(local_model.get_weights())
        
        K.clear_session()
    p, avg = sum_scaled_weights(scaled_local_weight_list, 10, unlab_dataset)
    p = p**2/(np.sum(p**2, axis = 1).reshape((len(p), 1)))

    #lab_dataset = tf.data.Dataset.from_tensor_slices((unlab_img_data,p)).batch(128)
    mlp_global = MLP()
    global_model = mlp_global.build()
    global_model.compile(loss=loss, 
                      optimizer=SGD(lr=0.001,momentum=0.9), 
                      metrics=metrics)
    global_model.set_weights(avg)
    swa = SWA(start_epoch=5, 
          lr_schedule='cyclic', 
          swa_lr=0.0004,
          swa_lr2=0.001,
          swa_freq=5,
          batch_size=32,
          verbose=0)
    global_model.fit(u_lab, p, epochs = 10, verbose = 0, batch_size=128)

    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

comm_round: 0 | global_acc: 10.900% | global_loss: 2.3030264377593994
comm_round: 1 | global_acc: 39.590% | global_loss: 2.2345852851867676
comm_round: 2 | global_acc: 52.050% | global_loss: 2.1880578994750977
comm_round: 3 | global_acc: 52.510% | global_loss: 2.170366048812866
comm_round: 4 | global_acc: 56.500% | global_loss: 2.131707191467285
comm_round: 5 | global_acc: 61.040% | global_loss: 2.115774631500244
comm_round: 6 | global_acc: 60.010% | global_loss: 2.1074087619781494
comm_round: 7 | global_acc: 64.590% | global_loss: 2.086677312850952
comm_round: 8 | global_acc: 64.670% | global_loss: 2.09403133392334
comm_round: 9 | global_acc: 65.580% | global_loss: 2.0810585021972656
comm_round: 10 | global_acc: 65.400% | global_loss: 2.047136068344116
comm_round: 11 | global_acc: 63.230% | global_loss: 2.0874855518341064
comm_round: 12 | global_acc: 66.190% | global_loss: 2.0655770301818848
comm_round: 13 | global_acc: 67.280% | global_loss: 2.0035312175750732
comm_round: 14 | global